In [1]:
import numpy as np
import wget
import keras
import tensorflow as tf

In [3]:
link = 'https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5'
name = 'model.h5'
wget.download(link, name)

'model.h5'

In [4]:
model = keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite = converter.convert()

with open('lite_dino-dragon_model.tflite', 'wb') as f:
    f.write(tflite)

INFO:tensorflow:Assets written to: C:\Users\Godwin\AppData\Local\Temp\tmp91j7zpr6\assets


INFO:tensorflow:Assets written to: C:\Users\Godwin\AppData\Local\Temp\tmp91j7zpr6\assets


In [7]:
!dir 

 Volume in drive C has no label.
 Volume Serial Number is D017-49E3

 Directory of c:\Users\Godwin\Documents\Workflow\MLZoomcamp\ML-ZoomCamp\Week9\Assignment

11/26/2022  09:11 PM    <DIR>          .
11/26/2022  09:11 PM    <DIR>          ..
11/26/2022  08:51 PM                 0 dino.ipynb
11/26/2022  09:07 PM        44,866,200 lite_dino-dragon_model.tflite
11/26/2022  09:07 PM        89,759,168 model.h5
               3 File(s)    134,625,368 bytes
               2 Dir(s)  17,338,519,552 bytes free


In [8]:
interpreter = tf.lite.Interpreter(model_path = 'lite_dino-dragon_model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [9]:
print(input_index, output_index)

0 13


In [10]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [17]:
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'

img = download_image(url)
img = prepare_image(img, (150,150))

C:\Users\Godwin\AppData\Local\Temp\ipykernel_5564\2487457277.py:17: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


In [18]:
img = np.array(img, dtype = 'float32')
img = np.array([img])
img = img/255.

In [19]:
img[0][0][0][0]

0.5529412

In [21]:
interpreter.set_tensor(input_index, img)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)
preds[0]

array([0.8244853], dtype=float32)